<a href="https://colab.research.google.com/github/paxcalpt/GoogleColab/blob/master/eLife_AuthorDomainStats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hi, Ricardo Henriques over here, I made this bit of code to pull the email domain from the corresponding authors in the eLife papers database. This was mostly out of personal curiosity. Please consider this code wildly untested or valdated.

Feel free to use the code as you think best...

In [15]:
!pip install termplotlib
!pip install maxminddb-geolite2
!git clone https://github.com/elifesciences/elife-article-xml.git
domains = {}

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 26.1 MB 1.9 MB/s 
     |████████████████████████████████| 330 kB 35.3 MB/s 
  Created wheel for maxminddb-geolite2: filename=maxminddb_geolite2-2018.703-py3-none-any.whl size=26407052 sha256=aa18a75c60563d23313ad259297b10fab132d35c0eda501fb651f4281a843672
  Stored in directory: /root/.cache/pip/wheels/21/f8/f5/c3d091f638042b802aa461ae6087b6f4106df7bf9a46ad4114
  Created wheel for maxminddb: filename=maxminddb-2.2.0-py2.py3-none-any.whl size=16356 sha256=a280783a5a78831505174ae6c1077088e3b392a9cff749602426308cf6f61614
  Stored in directory: /root/.cache/pip/wheels/3b/eb/d9/b4407ec5d0ccddce61c8d2f4db72a68a77a04adea2245b98ef
Successfully built maxminddb-geolite2 maxminddb
fatal: destination path 'elife-article-xml' already exists and is not an empty direc

In [34]:
import glob, termplotlib, socket
from geolite2 import geolite2

greader = geolite2.reader()  
files_xml = glob.glob("elife-article-xml/articles/*v1.xml")
#print(files_xml)
emailServers = {}
countries = {}

doCountries = True # Change this to False if you want faster calculation

counter = 1
nFiles = len(files_xml)
for fname in files_xml:
  print(f"Processing {counter}/{nFiles} ({round(100.*counter/nFiles)}%): {fname}")
  counter += 1
  with open(fname) as f:
    txt = f.read()
    #print(txt)
    #break

    ref = 0
    servers = []

    while 1:
      try:
        first = "<email>"
        last = "</email>"
        s = txt[ref:]
        start = s.index(first) + len(first)
        end = s.index(last, start)
        server = s[start:end].split("@")[1]
        if server != "gmail.com":
          servers.append(server)
        ref = ref+end
      except ValueError: 
        break
    
    for server in servers:
      if doCountries:
        if server in domains:
          country = domains[server]
        else:
          #try:
          try:
            ip = socket.gethostbyname(server)
            v = greader.get(ip)
            country = v['country']['iso_code']
            domains[server] = country
            print(v)
          except Exception: 
            country = "Unknown"   

        if country in countries:
          countries[country] += 1
        else:
          countries[country] = 1

      if server in emailServers:
        emailServers[server] += 1
      else:
        emailServers[server] = 1

print("\nTop domains:")
keys_server = sorted(emailServers.keys(), key=lambda k: emailServers[k], reverse=True)
for key in keys_server[:20]:
  print(f"{key}: {emailServers[key]}")

print("\nTop countries:")
keys_country = sorted(countries.keys(), key=lambda k: countries[k], reverse=True)
for key in keys_country[:20]:
  print(f"{key}: {countries[key]}")





Streaming output truncated to the last 5000 lines.
Processing 8689/13644 (64%): elife-article-xml/articles/elife-31220-v1.xml
Processing 8690/13644 (64%): elife-article-xml/articles/elife-01381-v1.xml
Processing 8691/13644 (64%): elife-article-xml/articles/elife-66588-v1.xml
Processing 8692/13644 (64%): elife-article-xml/articles/elife-50850-v1.xml
Processing 8693/13644 (64%): elife-article-xml/articles/elife-40655-v1.xml
Processing 8694/13644 (64%): elife-article-xml/articles/elife-06003-v1.xml
Processing 8695/13644 (64%): elife-article-xml/articles/elife-69288-v1.xml
Processing 8696/13644 (64%): elife-article-xml/articles/elife-46764-v1.xml
Processing 8697/13644 (64%): elife-article-xml/articles/elife-68035-v1.xml
Processing 8698/13644 (64%): elife-article-xml/articles/elife-67516-v1.xml
Processing 8699/13644 (64%): elife-article-xml/articles/elife-02208-v1.xml
Processing 8700/13644 (64%): elife-article-xml/articles/elife-50401-v1.xml
Processing 8701/13644 (64%): elife-article-xml/ar

In [36]:
import numpy as np
fig = termplotlib.figure()

count = np.asarray([emailServers[key] for key in keys_server])
keys_server = list(keys_server)
fig.barh(count, keys_server)
fig.show()

berkeley.edu                      [269]  ████████████████████████████████████████
ucsf.edu                          [247]  ████████████████████████████████████▊
ucl.ac.uk                         [200]  █████████████████████████████▊
yale.edu                          [194]  ████████████████████████████▉
cam.ac.uk                         [185]  ███████████████████████████▌
utsouthwestern.edu                [173]  █████████████████████████▊
hms.harvard.edu                   [163]  ████████████████████████▎
ucsd.edu                          [159]  ███████████████████████▋
mit.edu                           [148]  ██████████████████████
nih.gov                           [146]  █████████████████████▊
jhmi.edu                          [141]  █████████████████████
umich.edu                         [137]  ████████████████████▍
uw.edu                            [132]  ███████████████████▋
wustl.edu                         [131]  ███████████████████▌
uchicago.edu                      [122]  ██████